# **Импорт необходимых библиотек**

In [1]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import requests 
from urllib.parse import urlencode 
import re
from calendar import day_name

%matplotlib inline
# импорт библиотеки подключения к яндекс-метрике
from tapi_yandex_metrika import YandexMetrikaStats
from tapi_yandex_metrika import YandexMetrikaLogsapi
from pandas.io.json import json_normalize
import json
from requests_html import HTMLSession
session = HTMLSession()
from pandas import Series,DataFrame
from io import BytesIO
from time import sleep
import sys

In [2]:
# импорт библиотеки ИИ
import sketch

#примеры использования
#df.sketch.ask("Which columns are integer type?")
#df.sketch.howto("Plot the sales versus time")


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

## Ссылка на спецификации запросов Метрики

https://yandex.ru/dev/metrika/doc/api2/logs/intro.html

https://github.com/pavelmaksimov/tapi-yandex-metrika/blob/master/docs/logsapi.md

## Подготовка запроса

In [4]:
# токен доступа к метрике 
ACCESS_TOKEN = "!!!! TOKEN!!!! " 

# номер счетчика
count_number = "61617364"

client = YandexMetrikaLogsapi(
    access_token=ACCESS_TOKEN,
    default_url_params={'counterId': count_number})

In [5]:
# ВЫВОД ТЕКУЩИХ ЛОГОВ
result = client.allinfo().get()
print(result)

# ЦИКЛ ОЧИСТКИ ТЕКУЩИХ ЗАПРОСОВ
if len(result["requests"]) != 0:
    for i in range(0, len(result["requests"])):  # для того количества логов, которые есть в метритке
        req_id = result["requests"][i]["request_id"]   # получим номер очередного запроса
        res = client.clean(requestId=req_id).post()    # удалим этот запрос
        print(res)# распечатаем результат
else: print ("Логов на сервере нет")

<TapiClient object
{'requests': []}>
Логов на сервере нет


In [6]:
# запишем в переменную требуемые параметры
# дата, дата и время визита, ID визита, новый ли пользователь, страницы входа и выходаб, глубина,
# длительность, достигнутые цели, 
# источник трафика, рекламная система, кампания Директа, группа объявлений, объявление, 
# тип платформы, платформа, тип устройства 
#  1 — десктоп, 2 — мобильные телефоны, 3 — планшеты, 4 — TV
params = {
    "fields": "ym:s:date,ym:s:dateTime,ym:s:visitID,ym:s:clientID,ym:s:isNewUser,ym:s:startURL,\
ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID,ym:s:lastTrafficSource,\
ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,\
ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory",
    "source": "visits",
    "date1": "2022-12-01",
    "date2": "2022-12-31"
}

In [7]:
# Check the possibility of creating a report. Via HTTP GET method.
result = client.evaluate().get(params=params)
print(result)

<TapiClient object
{   'log_request_evaluation': {   'expected_size': 855742,
                                  'log_request_sum_max_size': 10737418240,
                                  'log_request_sum_size': 10737418240,
                                  'max_possible_day_quantity': 388972,
                                  'possible': True}}>


## Запрос

In [8]:
# Automatically download the report when it is prepared

client = YandexMetrikaLogsapi(
    access_token=ACCESS_TOKEN,
    default_url_params={'counterId': count_number},
    # Download the report when it will be created
    wait_report=True
)

info = client.create().post(params=params)
request_id = info["log_request"]["request_id"]

report = client.download(requestId=request_id).get()

data = report.data


print("Column names")
print(report.columns)


Column names
['ym:s:date', 'ym:s:dateTime', 'ym:s:visitID', 'ym:s:clientID', 'ym:s:isNewUser', 'ym:s:startURL', 'ym:s:endURL', 'ym:s:pageViews', 'ym:s:visitDuration', 'ym:s:goalsID', 'ym:s:lastTrafficSource', 'ym:s:lastAdvEngine', 'ym:s:lastDirectClickOrderName', 'ym:s:lastClickBannerGroupName', 'ym:s:lastDirectClickBannerName', 'ym:s:lastDirectPlatformType', 'ym:s:lastDirectPlatform', 'ym:s:deviceCategory']


## Преобразование в датафрейм

In [9]:
# Преобразование ответа Метрики в датафрейм   ()      
file = open("cashe.csv", "w", encoding="utf-8")   # сздаем и открываем новый файл
file.write(data)            # записываем в него ответ API
file.close()                    # закрываем файл
df = pd.read_csv("cashe.csv", header=0, sep='\t', index_col=0, encoding='utf-8')     # записываем в датафрейм файл
df = df.reset_index()
df.head()

,ym:s:date,ym:s:dateTime,ym:s:visitID,ym:s:clientID,ym:s:isNewUser,ym:s:startURL,ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID,ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory
0,2022-12-02,2022-12-02 13:01:31,7166621577389277251,1669975291682516483,1,https://kassa.life/arenda,https://kassa.life/arenda,1,0,[],ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса с эквайрингом в аренду за 1400 ₽/мес. Все включено,Context,dsp-opera-browser.yandex.ru,2
1,2022-12-02,2022-12-02 07:11:58,7161123543182737712,1669954321837598400,1,https://kassa.life/arenda,https://kassa.life/arenda,1,15,[],ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,com.lyxoto.master.forminecraftpe,2
2,2022-12-21,2022-12-21 10:21:19,7594437464571772991,1671607278486843246,1,"https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145","https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145",1,0,[],ad,ya_undefined,0,NaN,0,NaN,NaN,2
3,2022-12-21,2022-12-21 10:43:44,7594789944198955108,1671608623254784067,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/tilda/click/rec487399334/button2,2,77,[202896550],ad,ya_direct,ФН / МК / Владимир / Целевые / CPC: 3к/нед - 14.12,ФН / МК / Владимир / Целевые / CPC: 3к/нед - 14.12,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,com.EmZi.BubbleKlass,2
4,2022-12-18,2022-12-18 18:32:58,7534222670115373374,1671377576702224136,1,https://kassa.life/evotor5,https://kassa.life/evotor5,1,0,[],ad,ya_direct,Кассы / Эвотор 5 / МК,Кассы / Эвотор 5 / МК,"Онлайн-касса Эвотор 5 за 9900 руб! Б/у, «как новая»",Context,com.europosit.pixelcoloring,2


In [10]:
# ВЫВОД ТЕКУЩИХ ЛОГОВ
result = client.allinfo().get()
print(result)

# ЦИКЛ ОЧИСТКИ ТЕКУЩИХ ЗАПРОСОВ
if len(result["requests"]) != 0:
    for i in range(0, len(result["requests"])):  # для того количества логов, которые есть в метритке
        req_id = result["requests"][i]["request_id"]   # получим номер очередного запроса
        res = client.clean(requestId=req_id).post()    # удалим этот запрос
        print(res)# распечатаем результат
else: print ("Логов на сервере нет")

<TapiClient object
{   'requests': [   {   'attribution': 'LASTSIGN',
                        'counter_id': 61617364,
                        'date1': '2022-12-01',
                        'date2': '2022-12-31',
                        'fields': [   'ym:s:date',
                                      'ym:s:dateTime',
                                      'ym:s:visitID',
                                      'ym:s:clientID',
                                      'ym:s:isNewUser',
                                      'ym:s:startURL',
                                      'ym:s:endURL',
                                      'ym:s:pageViews',
                                      'ym:s:visitDuration',
                                      'ym:s:goalsID',
                                      'ym:s:lastTrafficSource',
                                      'ym:s:lastAdvEngine',
                                      'ym:s:lastDirectClickOrderName',
                                      'ym:s:

## *Функция группировки данных по одному параметру (group_df)*

In [68]:
def group_df(df, column):
    # сгруппируем 
    #
    grouped_device = df.groupby(column).agg({'ym:s:visitID': ['count'], 'ym:s:clientID': ['count', 'nunique'], 
                                             'ym:s:isNewUser' : ['mean'], 'ym:s:pageViews' : ['mean'], 
                                             'ym:s:visitDuration' : ['mean']}).reset_index()
    grouped_device.columns = [column, 'Достигнуто целей', 'Клиентов', 'Уник кл-в', 'Доля новых кл-в, %', 
                              'Просм страниц', 'Ср длит визита']
    
    grouped_device['Доля новых кл-в, %'] = grouped_device['Доля новых кл-в, %']  * 100
    for col in ('Доля новых кл-в, %', 'Просм страниц', 'Ср длит визита'):
        grouped_device[col] = round((grouped_device[col]), 2)
    
    return(grouped_device)

## *Функция группировки по двум пар-м (group_df_2_param)*

In [69]:
def group_df_2_param(df, column1, column2):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby([column1, column2] ).agg({'ym:s:visitID': ['count'], 'ym:s:clientID': ['count', 'nunique'], 
                                             'ym:s:isNewUser' : ['mean'], 'ym:s:pageViews' : ['mean'], 
                                             'ym:s:visitDuration' : ['mean']}).reset_index()
    grouped_device.columns = [column1, column2, 'Достигрунто целей', 'Клиентов', 'Уник кл-в', 'Доля новых кл-в, %', 
                              'Просм страниц', 'Ср длит визита']
    
    grouped_device['Доля новых кл-в, %'] = grouped_device['Доля новых кл-в, %']  * 100
    for col in ('Доля новых кл-в, %', 'Просм страниц', 'Ср длит визита'):
        grouped_device[col] = round((grouped_device[col]), 2)
        
    return(grouped_device)

## Создание датафрейма с описанием целей

In [57]:
data = {'Target_name': ["Телефон", "Мессенджер", "Подписка", "Заказ", "Отправка формы", "Оплата", "Спасибо Кассы", 
                   "Спасибо ФН", "Спасибо Аренда", "Спасибо сборная", "Не достигнута"], 'Target': [202896550, 202896559, 117180364, 
                                                                                 117184408, 151150963, 202896568, 
                                                                                 260187749, 260490669, 280525132, 280525412, 0]} 
 
# Create DataFram
targets_name = pd.DataFrame(data) 
targets_name

,Target_name,Target
0,Телефон,202896550
1,Мессенджер,202896559
2,Подписка,117180364
3,Заказ,117184408
4,Отправка формы,151150963
5,Оплата,202896568
6,Спасибо Кассы,260187749
7,Спасибо ФН,260490669
8,Спасибо Аренда,280525132
9,Спасибо сборная,280525412


# **Предобработка данных**

In [14]:
df_2 = df.copy()
# сделаем в колонке целей отдельные записи разделив по запятой
df_2.loc[:, 'ym:s:goalsID'] = df_2['ym:s:goalsID'].str.split(',')


Сначала посмотрим на типы данных и пропуски в даных

In [15]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ym:s:date                       2550 non-null   object
 1   ym:s:dateTime                   2550 non-null   object
 2   ym:s:visitID                    2550 non-null   int64 
 3   ym:s:clientID                   2550 non-null   uint64
 4   ym:s:isNewUser                  2550 non-null   int64 
 5   ym:s:startURL                   2550 non-null   object
 6   ym:s:endURL                     2550 non-null   object
 7   ym:s:pageViews                  2550 non-null   int64 
 8   ym:s:visitDuration              2550 non-null   int64 
 9   ym:s:goalsID                    2550 non-null   object
 10  ym:s:lastTrafficSource          2550 non-null   object
 11  ym:s:lastAdvEngine              2550 non-null   object
 12  ym:s:lastDirectClickOrderName   2550 non-null   

Необходимо:
- столбец даты привести к формату даты
- посмотреть что с пропусками в столбцах имени группы, типе платформы

In [131]:
# преобразование в тип даты
df_2['ym:s:date'] = pd.to_datetime(df_2['ym:s:date'])
df_2['ym:s:dateTime'] = pd.to_datetime(df_2['ym:s:dateTime'])

In [17]:
# посмотрим на строки, где нет информации по имени группы объявлений и неопределен источник перехода
df_2[(df_2['ym:s:lastClickBannerGroupName'].isna()  & (df_2['ym:s:lastAdvEngine'] == "ya_undefined"))]

,ym:s:date,ym:s:dateTime,ym:s:visitID,ym:s:clientID,ym:s:isNewUser,ym:s:startURL,ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID,ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory
2,2022-12-21,2022-12-21 10:21:19,7594437464571772991,1671607278486843246,1,"https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145","https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145",1,0,[[]],ad,ya_undefined,0,NaN,0,NaN,NaN,2
10,2022-12-28,2022-12-28 19:40:32,7761777888459227170,1672245632498381876,1,https://kassa.life/,https://kassa.life/,1,15,[[]],organic,ya_undefined,0,NaN,0,NaN,NaN,2
14,2022-12-03,2022-12-03 01:29:12,7178381446900350998,16700201511021129930,1,https://kassa.life/arenda,https://kassa.life/arenda,1,15,[[]],ad,ya_undefined,0,NaN,0,NaN,NaN,2
19,2022-12-06,2022-12-06 23:03:50,7266692224077529126,166999217386939184,0,https://kassa.life/arenda,https://kassa.life/arenda,1,15,[[]],ad,ya_undefined,0,NaN,0,NaN,NaN,2
21,2022-12-26,2022-12-26 01:35:51,7699418579705004246,1672007736891305210,1,https://kassa.life/arenda,https://kassa.life/arenda,1,15,[[]],ad,ya_undefined,0,NaN,0,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,2022-12-02,2022-12-02 07:12:13,7161127526140739843,1669954333508038145,1,https://kassa.life/arenda,https://kassa.life/arenda,1,18,[[]],ad,ya_undefined,0,NaN,0,NaN,NaN,2
2542,2022-12-19,2022-12-19 03:55:57,7543077543719731276,1671411364330079009,1,https://kassa.life/evotor5,https://kassa.life/evotor5,1,0,[[]],ad,ya_undefined,0,NaN,0,NaN,NaN,2
2543,2022-12-26,2022-12-26 17:22:45,7714312125904060722,1642412129750261337,0,https://kassa.life/1,https://kassa.life/,2,16,[[]],direct,ya_undefined,0,NaN,0,NaN,NaN,1
2544,2022-12-23,2022-12-23 20:48:23,7649598733636075839,1671612728833923469,0,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,2,1137,[[]],ad,ya_undefined,0,NaN,0,NaN,NaN,2


728 из 743 отстутстующих значений в столбце наименования баннера это те, где Метрикой не определен источник перехода. Заменим отсутствующие значения на слово "не определен

In [18]:
df_2['ym:s:lastClickBannerGroupName'] = df_2['ym:s:lastClickBannerGroupName'].fillna('Не определен')

In [20]:
# посмотрим какие значения есть в столбце типа платформы
df_2['ym:s:lastDirectPlatformType'].unique()

array(['Context', nan, 'Search'], dtype=object)

In [21]:
# посмотрим какие значения есть в столбце типа платформы
df_2['ym:s:lastDirectPlatform'].unique()

array(['dsp-opera-browser.yandex.ru', 'com.lyxoto.master.forminecraftpe',
       nan, 'com.EmZi.BubbleKlass', 'com.europosit.pixelcoloring',
       'infox.sg', 'com.crone.babyskins', 'game.yandex.ru', 'dzen.ru',
       'com.pronetis.ironball2', 'dsp-ironsource.yandex.ru',
       'com.xenonstudio.serversforminecraft',
       'com.openmygame.games.android.jigsawpuzzle',
       'dsp-vungle.yandex.ru', 'yandex.ru', 'bing.com', 'turokser.online',
       'rbc.ru', 'урок.рф', 'm.video.yandex.ru', 'com.seenax.HideAndSeek',
       'dsp-start-io.yandex.ru', 'my.mail.ru', 'muzofond.fm', 'mail.ru',
       'com.al.realdrive', 'dsp-huawei.yandex.ru',
       'com.yandex.alicekit.demo', 'ru.inkout.mcko',
       'com.openmygame.games.android.wordpizza', 'otzovik.com',
       'trashbox.ru', 'www.alpha-ag.ru', 'blitz.plus', 'win.mail.ru',
       'com.vpn-buck.proxy', 'dsp-inneractive.yandex.ru',
       'disk.yandex.ru', 'com.proto.MonsterHorde', 'dsp.yandex.ru',
       'com.lemonsquad.simulatorbs', 'com.

In [22]:
df_2[df_2['ym:s:lastDirectPlatformType'].isna()]

,ym:s:date,ym:s:dateTime,ym:s:visitID,ym:s:clientID,ym:s:isNewUser,ym:s:startURL,ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID,ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory
2,2022-12-21,2022-12-21 10:21:19,7594437464571772991,1671607278486843246,1,"https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145","https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145",1,0,[[]],ad,ya_undefined,0,Не определен,0,NaN,NaN,2
5,2022-12-02,2022-12-02 06:00:04,7159992619298717922,1669950002961104824,1,https://kassa.life/arenda,https://kassa.life/arenda,1,0,[[]],ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Никаких лишних доплат!,NaN,NaN,2
10,2022-12-28,2022-12-28 19:40:32,7761777888459227170,1672245632498381876,1,https://kassa.life/,https://kassa.life/,1,15,[[]],organic,ya_undefined,0,Не определен,0,NaN,NaN,2
12,2022-12-29,2022-12-29 00:09:55,7766014910242226358,1672261803719003038,1,https://kassa.life/evotor5,https://kassa.life/evotor5,1,15,[[]],ad,ya_direct,Кассы / Эвотор 5 / МК — Лучшая цена — 1600р,Кассы / Эвотор 5 / МК — Лучшая цена — 1600р,Эвотор 5 (б/у) за 9900 руб! Лучшее предложение на рынке!,NaN,NaN,2
13,2022-12-20,2022-12-20 14:14:37,7575457734879608894,1671534877666733199,1,https://kassa.life/zamena-fn?block=premium.3&source=search.none,https://kassa.life/zamena-fn?block=premium.3&source=search.none,1,0,[[]],ad,ya_direct,ФН / МК / СПб / Только целевые запросы — 2100,ФН / МК / СПб / Только целевые запросы — 2100,Фискальный накопитель на 16 и 36 мес. Всего от 9400 руб!,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,2022-12-02,2022-12-02 07:12:13,7161127526140739843,1669954333508038145,1,https://kassa.life/arenda,https://kassa.life/arenda,1,18,[[]],ad,ya_undefined,0,Не определен,0,NaN,NaN,2
2542,2022-12-19,2022-12-19 03:55:57,7543077543719731276,1671411364330079009,1,https://kassa.life/evotor5,https://kassa.life/evotor5,1,0,[[]],ad,ya_undefined,0,Не определен,0,NaN,NaN,2
2543,2022-12-26,2022-12-26 17:22:45,7714312125904060722,1642412129750261337,0,https://kassa.life/1,https://kassa.life/,2,16,[[]],direct,ya_undefined,0,Не определен,0,NaN,NaN,1
2544,2022-12-23,2022-12-23 20:48:23,7649598733636075839,1671612728833923469,0,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,2,1137,[[]],ad,ya_undefined,0,Не определен,0,NaN,NaN,2


Какой-то логики в появлении пропусков не видно, похоже просто на потерю данных. Заполним пропуски следующим образом: там где источник трафика реклама укажем "Context", там где organic укажем "search", в остальных случаях укажем "не определен".
В столбце "ym:s:lastDirectPlatform" просто укажем "не определен"

In [23]:
# заполнение данных по платформе
df_2['ym:s:lastDirectPlatform'] = df_2['ym:s:lastDirectPlatform'].fillna('Неизвестен')

In [24]:
# заполнение данных по типу платформы в зависимости от значения столбца источника трафика
df_2.loc[(df['ym:s:lastDirectPlatformType'].isna()) & (df_2['ym:s:lastTrafficSource'] == 'direct'), 'ym:s:lastDirectPlatformType'] = df_2['ym:s:lastDirectPlatformType'].fillna('Context')
df_2.loc[(df['ym:s:lastDirectPlatformType'].isna()) & (df_2['ym:s:lastTrafficSource'] == 'organic'), 'ym:s:lastDirectPlatformType'] = df_2['ym:s:lastDirectPlatformType'].fillna('Search')
df_2.loc[(df['ym:s:lastDirectPlatformType'].isna()), 'ym:s:lastDirectPlatformType'] = df_2['ym:s:lastDirectPlatformType'].fillna(df_2['ym:s:lastTrafficSource'])


In [58]:
# сделаем копию датафрейма для избежания ошибок 
df_with_goals = df_2.copy()

# разделим цели на отдельные строки через функцию explode
df_split_target = df_with_goals.explode('ym:s:goalsID').reset_index()

# удалим символы [ ] в начале и конце строки
df_split_target['ym:s:goalsID'] = df_split_target['ym:s:goalsID'].map(lambda x: x.lstrip("[ ").rstrip("] "))

# заменим пропуски значений нулями для дальнейшего корректного преобразования в целое число
df_split_target['ym:s:goalsID'] = df_split_target['ym:s:goalsID'].replace('', 0)

# и приведем цели к целочисленному типу
df_split_target['ym:s:goalsID'] = df_split_target['ym:s:goalsID'].astype('int')

In [59]:
# уберем дубликатыв в датафрейме, образовавшиеся после деления на строки списка целей
# так как нам не интересны одинаковые цели, достигнутые в ходе одного визита
df_split_target = df_split_target.drop_duplicates().reset_index()
df_split_target = df_split_target.drop(['level_0', 'index'], axis=1)

# **Исследовательский анализ данных**

## Присоединение названия целей

Присоединим к этому получившемуся датафрейму названия целей

In [61]:
df_split_target = df_split_target.merge(targets_name, left_on = 'ym:s:goalsID', right_on = 'Target')

Сейчас у нас в датафрейме только уникальные визиты, приведшие к достижению одной из целей (исключены визиты, в ходе которых одна цель была достигнута несколько раз), а также визиты, которые не привели к достижению цели.

## Частота достижения целей

In [158]:
# посмотрим на то, сколько раз была достигнута та или иная цель
df_by_target = group_df(df_split_target, 'Target_name')
df_by_target

,Target_name,Достигнуто целей,Клиентов,Уник кл-в,"Доля новых кл-в, %",Просм страниц,Ср длит визита
0,Мессенджер,41,41,39,92.68,3.34,290.78
1,Не достигнута,2387,2387,1873,76.04,1.27,47.32
2,Отправка формы,56,56,54,82.14,3.25,182.48
3,Спасибо Кассы,8,8,8,62.50,9.12,798.75
4,Спасибо ФН,18,18,17,77.78,3.89,124.33
5,Телефон,65,65,60,73.85,3.43,320.38


## Частота целей в зависимости от источника трафика

In [159]:
df_by_target_2 = group_df_2_param(df_split_target,'ym:s:lastTrafficSource', 'Target_name')
df_by_target_2[(df_by_target_2['ym:s:lastTrafficSource'] == "ad") | (df_by_target_2['ym:s:lastTrafficSource'] == "organic")]

,ym:s:lastTrafficSource,Target_name,Достигрунто целей,Клиентов,Уник кл-в,"Доля новых кл-в, %",Просм страниц,Ср длит визита
0,ad,Мессенджер,23,23,22,95.65,2.17,277.70
1,ad,Не достигнута,2057,2057,1663,78.76,1.18,39.50
2,ad,Отправка формы,52,52,50,82.69,3.12,171.50
3,ad,Спасибо Кассы,7,7,7,57.14,9.29,790.71
4,ad,Спасибо ФН,15,15,14,86.67,4.00,132.13
5,ad,Телефон,39,39,37,71.79,3.08,281.41
15,organic,Мессенджер,12,12,12,100.00,5.83,358.50
16,organic,Не достигнута,83,83,75,80.72,2.33,95.67
17,organic,Отправка формы,3,3,3,66.67,6.00,431.00
18,organic,Спасибо Кассы,1,1,1,100.00,8.00,855.00


По данным декабря 2022 года видно:
- клиенты. пришедшие по рекламе чаще всего заполняют и отправляют форму, на втором месте телефон, на третьем мессенджеры
- клиенты пришедщие на сайт органическим путем чаще смотрят номер телефона, обращаются в мессенджер, но не заполняют форму
- можно предположить, что клиенты, кликнувшие на рекламный баннер передают инициативу в обращении продавцу
- другое предположение, что те, кто приходит по самому популярному запросу - замена ФН, видят только возможность отправить форму, а те кто приходит через поиск также видят опции на странице касс, например.
- клиенты, пришедшие через поиск проводят на сайте больше времени и более глубоко его смотрят

## Типичное поведение посетителя 

In [160]:
# отсортируем датафрейм по пользователю и внутри пользователя по дате и времени визита
df_split_target = df_split_target.sort_values(by = ['ym:s:clientID', 'ym:s:dateTime'])

In [161]:
df_split_target

,ym:s:date,ym:s:dateTime,ym:s:visitID,ym:s:clientID,ym:s:isNewUser,ym:s:startURL,ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID,ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory,Target_name,Target
523,2022-12-02,2022-12-02 07:44:51,7161640801694908696,0,1,https://kassa.life/,https://kassa.life/,1,0,0,ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,dsp-start-io.yandex.ru,2,Не достигнута,0
1000,2022-12-02,2022-12-02 12:37:57,7166250844517302430,0,1,https://kassa.life/,https://kassa.life/,1,0,0,ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,com.zm.watersort,2,Не достигнута,0
1042,2022-12-21,2022-12-21 11:40:17,7595679463281524963,0,1,https://kassa.life/,https://kassa.life/,1,0,0,ad,ya_direct,ФН / МК / Владимир / Целевые / CPC: 3к/нед - 14.12,ФН / МК / Владимир / Целевые / CPC: 3к/нед - 14.12,Купить фискальный накопитель 15/36. Всего от 9400 руб!,Context,com.robtopx.geometrydashworld,2,Не достигнута,0
943,2022-12-02,2022-12-02 07:35:42,7161496888595185874,16699557484636700,1,https://kassa.life/arenda,https://kassa.life/arenda,1,0,0,ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,hitmotop.com,2,Не достигнута,0
1182,2022-12-02,2022-12-02 08:44:44,7162582654467965190,16699598838451870,1,https://kassa.life/arenda,https://kassa.life/arenda,1,0,0,ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,wwv.zvuch.com,1,Не достигнута,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,2022-12-30,2022-12-30 11:38:24,7799492872071282874,16723895031050876850,1,https://kassa.life/arenda,https://kassa.life/kassa-success,8,855,151150963,organic,ya_undefined,0,Не определен,0,Search,Неизвестен,1,Отправка формы,151150963
2568,2022-12-30,2022-12-30 11:38:24,7799492872071282874,16723895031050876850,1,https://kassa.life/arenda,https://kassa.life/kassa-success,8,855,260187749,organic,ya_undefined,0,Не определен,0,Search,Неизвестен,1,Спасибо Кассы,260187749
1743,2022-12-30,2022-12-30 14:22:50,7802079305308045548,16723895031050876850,0,https://kassa.life/arenda,https://kassa.life/arenda,12,1408,0,organic,ya_undefined,0,Не определен,0,Search,Неизвестен,1,Не достигнута,0
1952,2022-12-30,2022-12-30 12:43:53,7800522888293122365,16723934331063898531,1,https://kassa.life/,https://kassa.life/kassa-v-kredit,2,110,0,organic,ya_undefined,0,Не определен,0,Search,Неизвестен,3,Не достигнута,0


In [162]:
# общее количество событий
user_count = df_split_target.groupby('Target_name').agg({'ym:s:clientID': 'nunique'}).reset_index().sort_values(by = 'ym:s:clientID', ascending = False)
user_count

,Target_name,ym:s:clientID
1,Не достигнута,1873
5,Телефон,60
2,Отправка формы,54
0,Мессенджер,39
4,Спасибо ФН,17
3,Спасибо Кассы,8


Общая воронка продаж для нас не информативна, так как собственно единая воронка на сайте отстутствует, клиент может нажать на телефон, но не отправлять форму, или совершить заказ, но не нажимать на телефон.

Соответственно нам требуется какой-то другой подход к анализу поведения клиентов.

In [163]:
# сформируем датафрейм только с новыми клиентами
new_users_df = df_split_target[df_split_target['ym:s:isNewUser'] == 1]

In [164]:
# отсортируем их по номеру клиента и внутри по времени события
new_users_df = new_users_df.sort_values(by = ['ym:s:clientID', 'ym:s:dateTime'])
# получим только первые события
first_event = new_users_df.groupby('ym:s:clientID')['ym:s:dateTime'].first().reset_index()

In [173]:
# сформируем датафрейм целевых событий
target_events = df_split_target[df_split_target['Target'] != 0]

In [174]:
target_events.head()

,ym:s:date,ym:s:dateTime,ym:s:visitID,ym:s:clientID,ym:s:isNewUser,ym:s:startURL,ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID,ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory,Target_name,Target
2457,2022-12-02,2022-12-02 12:19:12,7165955841290141797,16699727516907103,1,https://kassa.life/arenda,https://kassa.life/arenda,1,2,151150963,ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,soloby.ru,2,Отправка формы,151150963
2459,2022-12-09,2022-12-09 16:51:06,7328777336308105253,16705938648779362,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,151150963,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Отправка формы,151150963
2512,2022-12-09,2022-12-09 16:51:06,7328777336308105253,16705938648779362,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,260490669,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Спасибо ФН,260490669
2532,2022-12-09,2022-12-09 16:51:06,7328777336308105253,16705938648779362,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,202896559,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Мессенджер,202896559
2454,2022-12-07,2022-12-07 16:33:18,7283198825418981497,166999217386939184,0,https://kassa.life/arenda,https://kassa.life/arenda,5,51,151150963,ad,ya_undefined,0,Не определен,0,ad,Неизвестен,2,Отправка формы,151150963


In [167]:
# посмотрим, сколько максимально целей клиенты достигали в течении одного визита
# для этого сгруппируем кол-во достижений цели по визитам и посмотрим на ТОП этого получившегося списка
tar_count = target_events.groupby('ym:s:visitID').agg({'ym:s:goalsID': 'count'}).reset_index().sort_values(by = 'ym:s:goalsID', ascending = False)
tar_count.head()

,ym:s:visitID,ym:s:goalsID
54,7328777336308105253,3
151,7799492872071282874,3
71,7394824980945436932,3
30,7263132539483062592,3
18,7213558458263797805,3


In [168]:
# посчитаем, сколько раз клиенты достигали одну или более целей
tar_count.groupby('ym:s:goalsID')['ym:s:visitID'].count()

ym:s:goalsID
1    126
2     22
3      6
Name: ym:s:visitID, dtype: int64

Всего 6 раз клиенты достигали 3 цели за визит, 22 раза они достигали 2 целей за визит и 126 раз достигали одну цель.

На следующем этапе исследования посмотрим, как быстро после первого посещения клиент делает целевое действие.


In [175]:
# присоединим к данным о дате и времени первого входа на сайт для новых клиентов
# информацию о событиях этого клиента
first_event = first_event.merge(target_events, on = 'ym:s:clientID')

In [176]:
first_event.head()

,ym:s:clientID,ym:s:dateTime_x,ym:s:date_x,Target_name_x,ym:s:visitID_x,ym:s:goalsID_x,ym:s:date_y,ym:s:dateTime_y,ym:s:visitID_y,ym:s:isNewUser,ym:s:startURL,ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID_y,ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory,Target_name_y,Target
0,16699727516907103,2022-12-02 12:19:12,2022-12-02,Отправка формы,7165955841290141797,151150963,2022-12-02,2022-12-02 12:19:12,7165955841290141797,1,https://kassa.life/arenda,https://kassa.life/arenda,1,2,151150963,ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,soloby.ru,2,Отправка формы,151150963
1,16705938648779362,2022-12-09 16:51:06,2022-12-09,Отправка формы,7328777336308105253,151150963,2022-12-09,2022-12-09 16:51:06,7328777336308105253,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,151150963,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Отправка формы,151150963
2,16705938648779362,2022-12-09 16:51:06,2022-12-09,Отправка формы,7328777336308105253,151150963,2022-12-09,2022-12-09 16:51:06,7328777336308105253,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,260490669,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Спасибо ФН,260490669
3,16705938648779362,2022-12-09 16:51:06,2022-12-09,Отправка формы,7328777336308105253,151150963,2022-12-09,2022-12-09 16:51:06,7328777336308105253,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,202896559,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Мессенджер,202896559
4,16705938648779362,2022-12-09 16:51:06,2022-12-09,Спасибо ФН,7328777336308105253,260490669,2022-12-09,2022-12-09 16:51:06,7328777336308105253,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,151150963,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Отправка формы,151150963


In [177]:
# можно будет убрать на финальном этапе, эта предобработка сделана на первом этапе
first_event['ym:s:date_x'] = pd.to_datetime(first_event['ym:s:date_x'])
first_event['ym:s:date_y'] = pd.to_datetime(first_event['ym:s:date_y'])

In [178]:
# посмотрим на разницу между первым визитом на сайт и первым целевым действием
first_event['diff'] = first_event['ym:s:date_x'] - first_event['ym:s:date_y']

In [179]:
first_event.head()

,ym:s:clientID,ym:s:dateTime_x,ym:s:date_x,Target_name_x,ym:s:visitID_x,ym:s:goalsID_x,ym:s:date_y,ym:s:dateTime_y,ym:s:visitID_y,ym:s:isNewUser,ym:s:startURL,ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID_y,ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory,Target_name_y,Target,diff
0,16699727516907103,2022-12-02 12:19:12,2022-12-02,Отправка формы,7165955841290141797,151150963,2022-12-02,2022-12-02 12:19:12,7165955841290141797,1,https://kassa.life/arenda,https://kassa.life/arenda,1,2,151150963,ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,soloby.ru,2,Отправка формы,151150963,0 days
1,16705938648779362,2022-12-09 16:51:06,2022-12-09,Отправка формы,7328777336308105253,151150963,2022-12-09,2022-12-09 16:51:06,7328777336308105253,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,151150963,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Отправка формы,151150963,0 days
2,16705938648779362,2022-12-09 16:51:06,2022-12-09,Отправка формы,7328777336308105253,151150963,2022-12-09,2022-12-09 16:51:06,7328777336308105253,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,260490669,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Спасибо ФН,260490669,0 days
3,16705938648779362,2022-12-09 16:51:06,2022-12-09,Отправка формы,7328777336308105253,151150963,2022-12-09,2022-12-09 16:51:06,7328777336308105253,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,202896559,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Мессенджер,202896559,0 days
4,16705938648779362,2022-12-09 16:51:06,2022-12-09,Спасибо ФН,7328777336308105253,260490669,2022-12-09,2022-12-09 16:51:06,7328777336308105253,1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/zamena-fn-vladimir/,7,193,151150963,ad,ya_direct,ФН / МК / Владимир / Широкие - 1800р — 2,ФН / МК / Владимир / Широкие - 1800р — 2,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,dsp-ironsource.yandex.ru,2,Отправка формы,151150963,0 days


In [187]:
# посмотрим на то, с какой задержкой клиенты делают целевое действие
df_diff = first_event.groupby('diff')['ym:s:clientID'].nunique().reset_index()
df_diff

,diff,ym:s:clientID
0,-5 days,1
1,-1 days,1
2,0 days,136
3,1 days,1
4,5 days,1


Видим, что подавляющее большинство клиентов делает целевое действие в тот же день, что и впервые заходят на сайт. Что повышает необходимость иметь СТА на каждой странице сайта.

# **Вывод**

В целом исследование данных логов много новой информации не дало. Основной вывод - клиенты совершают целевое действие в тот же день, что и впервые заходят на сайт, то есть быстро принимают решение, а значит сайт необходимо делать таким образом, что бы клиент мог за один визит найти всю необходимую информацию и совершить целевое действие.

Рекомендация - продумать и реализовать через цели Метрики воронку покупательского пути, что бы можно было отслеживать их реальный путь.